In [1]:
import pandas as pd
import numpy as np
import openrouteservice as ors
from dotenv import load_dotenv
from pyonemap import OneMap
import os
import requests
import json
import time
from collections import namedtuple
load_dotenv()
import folium

In [20]:
#get mrt data
mrt_df = pd.read_csv('../../data/Isochrone_data/mrt_station_colours.csv', index_col=0)
mrt_df.head()
coords = mrt_df[['Latitude', 'Longitude']].values.tolist()
print(coords)
# run this to start OTP: java -Xmx6G -jar otp-2.5.0-shaded.jar --load /Users/ethan/Documents/GitHub/Public-Transportation-In-Singapore/Backend/otp/  --serve

# Sample API request to the OTP isochrone endpoint
response = requests.get("http://localhost:8080/otp/traveltime/isochrone?batch=true&location=52.499959,13.388803&time=2023-04-12T10:19:03%2B02:00&modes=WALK,TRANSIT&arriveBy=false&cutoff=30M17S")


[[1.293657725, 103.8550812], [1.31719912, 103.8923649], [1.298655224, 103.8461942], [1.308547984, 103.8890647], [1.307377817, 103.8627676], [1.38670059, 103.8906213], [1.311405293, 103.7786378], [1.371292266, 103.8923805], [1.378615451, 103.7579956], [1.316432612, 103.882906], [1.399281988, 103.9059619], [1.37300805, 103.9492651], [1.440588562, 103.8009905], [1.321038134, 103.9129479], [1.41684852, 103.9066508], [1.432521149, 103.7740746], [1.394493045, 103.9004925], [1.41738337, 103.8329799], [1.392079837, 103.8800296], [1.345633658, 103.9384925], [1.345515306, 103.938437], [1.356191483, 103.9546345], [1.335145642, 103.8883361], [1.408452419, 103.8985585], [1.320065558, 103.8260244], [1.302812468, 103.8753377], [1.330857645, 103.7969068], [1.380298287, 103.7452918], [1.405323319, 103.8972905], [1.300264167, 103.8390858], [1.343197079, 103.9533772], [1.350633615, 103.8727717], [1.391694626, 103.8954847], [1.340469001, 103.8467991], [1.320440791, 103.8438256], [1.3131403, 103.8547547], 

Write a function for fetching isochrome. Configure params here

In [21]:
def fetch_isochrone(lat, lon, modes, cutoff="20M01S"):
    """Fetch isochrone GeoJSON for a given location."""
    url = "http://localhost:8080/otp/traveltime/isochrone"
    params = {
        "batch": "false",
        "location": f"{lat},{lon}",
        "time": "2024-04-02T10:00:00+02:00",
        "modes": modes,
        "cutoff": cutoff,
        "maxWalkDistance": 2,
        "walkReluctance": 4
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch isochrone for {lat},{lon}. Status code: {response.status_code}")
        return None
x = fetch_isochrone(1.298655224, 103.8461942, modes = "TRANSIT")
features = x['features'][0]['geometry']['coordinates'][0][0]
print(features)

[[103.836045, 1.294619], [103.837223, 1.29545], [103.838233, 1.295008], [103.839022, 1.29469], [103.839409, 1.294385], [103.840007, 1.293998], [103.840426, 1.293603], [103.840821, 1.292805], [103.840975, 1.292353], [103.841198, 1.292199], [103.841572, 1.291151], [103.84262, 1.290512], [103.842666, 1.290447], [103.844419, 1.290612], [103.84487, 1.290852], [103.846218, 1.290415], [103.846227, 1.290409], [103.846342, 1.290401], [103.846218, 1.290377], [103.845601, 1.288602], [103.845596, 1.287979], [103.846218, 1.287716], [103.847753, 1.288337], [103.848017, 1.288403], [103.848132, 1.288602], [103.848343, 1.288927], [103.848352, 1.290401], [103.849817, 1.290792], [103.850405, 1.290989], [103.851616, 1.291194], [103.852966, 1.291751], [103.853415, 1.291974], [103.853791, 1.292199], [103.854688, 1.293472], [103.85503, 1.293998], [103.855214, 1.294696], [103.855365, 1.295797], [103.856599, 1.297181], [103.856597, 1.297595], [103.856472, 1.298853], [103.855214, 1.297816], [103.854979, 1.29915

In [24]:
#Plotting out Isochrone
import plotly.graph_objects as go
# Create a Plotly figure
fig = go.FigureWidget()
fig.layout.hovermode = 'closest'
default_linewidth = 2
highlighted_linewidth = 3
color_sequence = ['blue', 'green', 'red', 'orange', 'purple', 'yellow']

# Fetch and add isochrones for each MRT station
for index, (lat, lon) in enumerate(coords):
    mrt_name = mrt_df[(mrt_df['Latitude'] == lat) & (mrt_df['Longitude'] == lon)]['MRT.Name'].iloc[0]
    # Add isochrone as a trace
    color = color_sequence[index%len(color_sequence)]
    fig.add_trace(go.Scattermapbox(
        mode="markers",
        lon=[lon],
        lat=[lat],
        text = [mrt_name],
        marker=dict(size=10,color=color),
        selected=go.scattermapbox.Selected(marker = {"size":25}),
        unselected=go.scattermapbox.Unselected(marker = {"color":"grey", "size":5})
    ))

x = fetch_isochrone(1.298655224, 103.8461942, modes = "BUS", cutoff="40M")
features = x['features'][0]['geometry']['coordinates'][0][0]
# Assuming coordinates are in [lon, lat] format
lon_coords = [coord[0] for coord in features]
lat_coords = [coord[1] for coord in features]
fig.add_trace(go.Scattermapbox(
                    mode="lines",
                    lon=lon_coords,
                    lat=lat_coords,
                    name=f"Isochrone {lat}, {lon}",
                    line=dict(width=1, color=color),
                    fill = "toself",
                    marker=dict(size=0),
                    text=[mrt_name],  # Set text for hover
                    hoverinfo='text'
                ))

# Define layout for the map
fig.update_layout(
    clickmode='event+select',
    mapbox_style="carto-positron",
    mapbox_zoom=12,
    mapbox_center={"lat": coords[0][0], "lon": coords[0][1]},
    showlegend = False
)
fig.show()

DATA NEEDED:

Isochrones: 10min-60min, 1min interval (ideal) else 5min interval

1. BICYCLE
2. BUS+ WALK
3. RAIL+ WALK
4. TRANSIT + WALK
5. CAR

Set up function for saving all isochrones by transport mode

In [12]:
def fetch_all_isochrones(df, mode, interval):
    """Fetch and append bicycle isochrones for each MRT station and specified time intervals."""
    # Define the cutoff intervals (in seconds) - lets us choose 1, 5, or 10 minute intervals later on
    cutoff_intervals = [f"{x}M" for x in range(10, 61, interval)]
    
    # Iterate over the cutoff intervals
    for interval in cutoff_intervals:
        # Define a column name for the interval
        column_name = f"isochrone_{interval}"
        # Initialize the column with empty lists
        df[column_name] = None
        
        # Iterate over each row in the DataFrame
        for index, row in df.iterrows():
            # Fetch the isochrone GeoJSON for the current MRT station and cutoff interval
            isochrone_geojson = fetch_isochrone(row['Latitude'], row['Longitude'], modes=mode, cutoff=interval )
            
            # Check if the response is not None
            if isochrone_geojson:
                # Extract the coordinates of the first feature (assuming single isochrone feature per request)
                if 'features' in isochrone_geojson and len(isochrone_geojson['features']) > 0:
                    features = isochrone_geojson['features'][0]['geometry']['coordinates'][0][0]
                    # Assign the extracted features to the corresponding DataFrame cell
                    df.at[index, column_name] = features
                print(f"Fetched isochrone for interval {interval}, MRT: {row['MRT.Name']}")
            else:
                print(f"Failed to fetch isochrone for interval {interval}, MRT: {row['MRT.Name']}")
    
    return df

# Note: Ensure that mrt_df already exists and contains 'MRT.Name', 'Latitude', and 'Longitude' columns


In [ ]:
# Apply the function to the mrt_df DataFrame
# bicycle_isochrones = mrt_df.copy()
# bicycle_isochrones = fetch_all_isochrones(bicycle_isochrones, interval=5)
# bicycle_isochrones.to_json('../../data/Isochrone_data/bicycle_isochrones.json', orient='records', lines=True, force_ascii=False)

In [13]:
bus_isochrones = mrt_df.copy()
bus_isochrones = fetch_all_isochrones(bus_isochrones,mode="BUS,WALK", interval=5)
bus_isochrones.to_json('../../data/Isochrone_data/bus_isochrones.json', orient='records', lines=True, force_ascii=False)

Fetched isochrone for interval 10M, MRT: ESPLANADE MRT STATION
Fetched isochrone for interval 10M, MRT: PAYA LEBAR MRT STATION
Fetched isochrone for interval 10M, MRT: DHOBY GHAUT MRT STATION
Fetched isochrone for interval 10M, MRT: DAKOTA MRT STATION
Fetched isochrone for interval 10M, MRT: LAVENDER MRT STATION
Fetched isochrone for interval 10M, MRT: RENJONG LRT STATION
Fetched isochrone for interval 10M, MRT: DOVER MRT STATION
Fetched isochrone for interval 10M, MRT: HOUGANG MRT STATION
Fetched isochrone for interval 10M, MRT: PHOENIX LRT STATION
Fetched isochrone for interval 10M, MRT: ALJUNIED MRT STATION
Fetched isochrone for interval 10M, MRT: COVE LRT STATION
Fetched isochrone for interval 10M, MRT: PASIR RIS MRT STATION
Fetched isochrone for interval 10M, MRT: ADMIRALTY MRT STATION
Fetched isochrone for interval 10M, MRT: KEMBANGAN MRT STATION
Fetched isochrone for interval 10M, MRT: PUNGGOL POINT LRT STATION
Fetched isochrone for interval 10M, MRT: MARSILING MRT STATION
Fetch

In [14]:
mrt_isochrones = mrt_df.copy()
mrt_isochrones = fetch_all_isochrones(mrt_isochrones,mode="RAIL,SUBWAY,TRAM,MONORAIL,WALK", interval=5)
mrt_isochrones.to_json('../../data/Isochrone_data/mrt_isochrones.json', orient='records', lines=True, force_ascii=False)

Fetched isochrone for interval 10M, MRT: ESPLANADE MRT STATION
Fetched isochrone for interval 10M, MRT: PAYA LEBAR MRT STATION
Fetched isochrone for interval 10M, MRT: DHOBY GHAUT MRT STATION
Fetched isochrone for interval 10M, MRT: DAKOTA MRT STATION
Fetched isochrone for interval 10M, MRT: LAVENDER MRT STATION
Fetched isochrone for interval 10M, MRT: RENJONG LRT STATION
Fetched isochrone for interval 10M, MRT: DOVER MRT STATION
Fetched isochrone for interval 10M, MRT: HOUGANG MRT STATION
Fetched isochrone for interval 10M, MRT: PHOENIX LRT STATION
Fetched isochrone for interval 10M, MRT: ALJUNIED MRT STATION
Fetched isochrone for interval 10M, MRT: COVE LRT STATION
Fetched isochrone for interval 10M, MRT: PASIR RIS MRT STATION
Fetched isochrone for interval 10M, MRT: ADMIRALTY MRT STATION
Fetched isochrone for interval 10M, MRT: KEMBANGAN MRT STATION
Fetched isochrone for interval 10M, MRT: PUNGGOL POINT LRT STATION
Fetched isochrone for interval 10M, MRT: MARSILING MRT STATION
Fetch

In [ ]:
public_isochrones = mrt_df.copy()
public_isochrones = fetch_all_isochrones(public_isochrones,mode="TRANSIT,WALK", interval=5)
public_isochrones.to_json('../../data/Isochrone_data/public_isochrones.json', orient='records', lines=True, force_ascii=False)

In [ ]:
car_isochrones = mrt_df.copy()
car_isochrones = fetch_all_isochrones(car_isochrones,mode="CAR", interval=5)
car_isochrones.to_json('../../data/Isochrone_data/car_isochrones.json', orient='records', lines=True, force_ascii=False)

In [10]:
bicycle_isochrones = pd.read_json('../../data/Isochrone_data/bicycle_isochrones.json', orient='records', lines=True)
bicycle_isochrones.head()

,MRT.Name,Latitude,Longitude,isochrone_10M,isochrone_15M,isochrone_20M,isochrone_25M,isochrone_30M,isochrone_35M,isochrone_40M,isochrone_45M,isochrone_50M,isochrone_55M,isochrone_60M
0,ESPLANADE MRT STATION,1.293658,103.855081,"[[103.861, 1.284867], [103.861722, 1.286027], ...","[[103.841367, 1.289686], [103.841584, 1.289203...","[[103.831873, 1.288905], [103.832188, 1.288877...","[[103.888542, 1.29323], [103.888929, 1.293201]...","[[103.869582, 1.3302019999999999], [103.868636...","[[103.812166, 1.2781959999999999], [103.812398...","[[103.809332, 1.262772], [103.810599, 1.262453...","[[103.813108, 1.337934], [103.813677, 1.338025...","[[103.806029, 1.338051], [103.80538, 1.338502]...","[[103.806029, 1.338051], [103.80538, 1.338502]...","[[103.806029, 1.338051], [103.80538, 1.338502]..."
1,PAYA LEBAR MRT STATION,1.317199,103.892365,"[[103.889556, 1.307294], [103.889875, 1.307317...","[[103.877473, 1.325003], [103.876758, 1.324139...","[[103.901946, 1.2953999999999999], [103.903282...","[[103.858745, 1.2968709999999999], [103.860358...","[[103.863491, 1.287579], [103.864052, 1.287626...","[[103.872771, 1.279147], [103.873047, 1.279572...","[[103.89785, 1.378247], [103.897584, 1.379152]...","[[103.872771, 1.279147], [103.873047, 1.279572...","[[103.829497, 1.336703], [103.829566, 1.336401...","[[103.829843, 1.273725], [103.829869, 1.2737],...","[[103.829843, 1.273725], [103.829869, 1.2737],..."
2,DHOBY GHAUT MRT STATION,1.298655,103.846194,"[[103.833806, 1.29983], [103.833988, 1.299424]...","[[103.850876, 1.283103], [103.85223, 1.283638]...","[[103.819005, 1.292228], [103.819595, 1.291808...","[[103.872969, 1.296341], [103.874022, 1.295847...","[[103.810599, 1.283574], [103.811982, 1.283408...","[[103.810599, 1.26787], [103.810721, 1.267758]...","[[103.807995, 1.263234], [103.8088, 1.262982],...","[[103.892113, 1.341375], [103.891971, 1.342099...","[[103.891038, 1.348936], [103.89062, 1.348876]...","[[103.891038, 1.348936], [103.89062, 1.348876]...","[[103.891038, 1.348936], [103.89062, 1.348876]..."
3,DAKOTA MRT STATION,1.308548,103.889065,"[[103.876785, 1.303843], [103.876851, 1.303841...","[[103.86878, 1.304214], [103.869766, 1.303662]...","[[103.885931, 1.334911], [103.88592, 1.334783]...","[[103.857381, 1.291005], [103.858034, 1.290733...","[[103.891038, 1.348776], [103.890094, 1.34835]...","[[103.898297, 1.357111], [103.897485, 1.357569...","[[103.898754, 1.3683589999999999], [103.898747...","[[103.829973, 1.272056], [103.831668, 1.273447...","[[103.824297, 1.278972], [103.824471, 1.27813]...","[[103.824297, 1.278972], [103.824471, 1.27813]...","[[103.921507, 1.374358], [103.921501, 1.374475..."
4,LAVENDER MRT STATION,1.307378,103.862768,"[[103.866625, 1.300138], [103.867741, 1.300542...","[[103.863903, 1.28979], [103.864052, 1.289684]...","[[103.844063, 1.295915], [103.844374, 1.295568...","[[103.840038, 1.288104], [103.84054, 1.287061]...","[[103.830175, 1.290805], [103.830389, 1.29043]...","[[103.819432, 1.290856], [103.819595, 1.290773...","[[103.834251, 1.351184], [103.83387, 1.350467]...","[[103.902546, 1.342812], [103.902214, 1.343898...","[[103.806451, 1.273721], [103.80648, 1.273687]...","[[103.919824, 1.376512], [103.920608, 1.378072...","[[103.902546, 1.342812], [103.902214, 1.343898..."
